<a href="https://colab.research.google.com/github/insightcampus/202008-youth-bigdata/blob/master/hnjoo/topic_modeling/T.M_3_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [토픽 모델링 실습3] LDA: 네이버 영화 리뷰 토픽 모델링

### 0. 사전준비

In [ ]:
# 나눔폰트 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 런타임 다시 시작

In [ ]:
# 코렙 mecab 설치 
!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
# !sudo apt-get install python-dev; pip install konlpy     # Python 2.x
!sudo apt-get install python3-dev; pip3 install konlpy   # Python 3.x
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd 

/root


### 1. 데이터 전처리

In [4]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

In [8]:
import re
from konlpy.tag import Mecab
mecab = Mecab()

# 데이터 불러오기
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/NLP_edu/LDA_example.csv의 사본')

# 데이터 전처리 (결측치 확인)
  # 특수 문자 제거
remove_except_ko = re.compile(r"[^가-힣ㄱ-하-ㅣ\\s]")

def preprocess(text):
  text = re.sub(remove_except_ko,' ',text).strip()
  return text

df['clean_doc'] = df['content'].map(lambda x : preprocess(x))

In [9]:
mecab.pos(df['clean_doc'][0])

[('후불', 'NNG'),
 ('정산', 'NNG'),
 ('방식', 'NNG'),
 ('악용', 'NNG'),
 ('해', 'XSV+EC'),
 ('서버', 'NNG'),
 ('혹사', 'NNG'),
 ('시키', 'XSV'),
 ('고', 'EC'),
 ('잠', 'NNG'),
 ('적', 'XSN'),
 ('비트코인', 'NNP'),
 ('등', 'NNB'),
 ('가상', 'NNG'),
 ('통화', 'NNG'),
 ('광풍', 'NNG'),
 ('으로', 'JKB'),
 ('국내', 'NNG'),
 ('클라우드', 'NNP'),
 ('업계', 'NNG'),
 ('가', 'JKS'),
 ('피해', 'NNG'),
 ('를', 'JKO'),
 ('받', 'VV'),
 ('고', 'EC'),
 ('있', 'VX'),
 ('다', 'EC'),
 ('비트코인', 'NNP'),
 ('의', 'JKG'),
 ('채굴', 'NNG'),
 ('과정', 'NNG'),
 ('에서', 'JKB'),
 ('고', 'XPN'),
 ('성능', 'NNG'),
 ('그래픽', 'NNG'),
 ('카드', 'NNG'),
 ('등', 'NNB'),
 ('비싼', 'VA+ETM'),
 ('하드웨어', 'NNG'),
 ('가', 'JKS'),
 ('필요', 'NNG'),
 ('한데', 'XSA+EC'),
 ('이', 'MM'),
 ('부담', 'NNG'),
 ('을', 'JKO'),
 ('줄이', 'VV'),
 ('기', 'ETN'),
 ('위해', 'VV+EC'),
 ('클라우드', 'NNP'),
 ('서비스', 'NNG'),
 ('를', 'JKO'),
 ('통해', 'VV+EC'),
 ('가상', 'NNG'),
 ('통화', 'NNG'),
 ('를', 'JKO'),
 ('채굴', 'NNG'),
 ('하', 'XSV'),
 ('는', 'ETM'),
 ('변칙', 'NNG'),
 ('사례', 'NNG'),
 ('가', 'JKS'),
 ('증가', 'NNG'),
 ('하', 'XSV')

In [10]:
  # 토큰화 및 불용어 처리  
stop_word = ['께서','에서','이다','에게','으로','이랑','까지','부터','하다']
stopPos = 'JKS JKC JKG JKO JKB JKY JKQ JC JX SF SE SS SP SSO SSC SC SY SH SL SN MM MAG MAJ'.split()
  
def postagging_mecab(text):
  text = mecab.pos(text)
  text = [i for i in text if len(i)>1]
  text = [i for i in text if i[0] not in stop_word]
  text = [i[0] for i in text if i[1] not in stopPos]
  return text

df['clean_doc'] = df['clean_doc'].apply(lambda x : postagging_mecab(x))
tokenized_docs = df['clean_doc'].apply(lambda x: ' '.join([item for item in x]))

In [11]:
df.head()

,content,clean_doc
0,후불정산 방식 악용해 서버 혹사 시키고 잠적 비트코인 등 가상통화 광풍으로 국내 클...,"[후불, 정산, 방식, 악용, 해, 서버, 혹사, 시키, 고, 잠, 적, 비트코인,..."
1,개인정보 유출 공지 안내 이메일로 위장한 '가상화폐 채굴 악성코드'가 나타났다. P...,"[개인, 정보, 유출, 공지, 안내, 이메일, 위장, 한, 가상, 화폐, 채굴, 악..."
2,행정안전부 국가기록원은 9일부터 국민의 열람 수요가 많은 서울 경기권 지적원도 약 ...,"[행정, 국가, 기록원, 일, 국민, 열람, 수요, 많, 은, 서울, 경기, 권, ..."
3,"소비자잡지 초이스, 추천제품 선정 LG전자 건조기가 호주서 호평을 받았다.LG전자 ...","[소비자, 잡지, 초이스, 추천, 제품, 선정, 전자, 건조기, 호주, 호평, 받,..."
4,스타크래프트 AI대회 韓 첫 1위데이비드 처칠 David Churchill 캐나다 ...,"[스타크래프트, 대회, 韓, 위, 데이비드, 처칠, 캐나다, 메모리얼, 대학교, 컴..."


In [12]:
tokenized_docs

0     후불 정산 방식 악용 해 서버 혹사 시키 고 잠 적 비트코인 등 가상 통화 광풍 국...
1     개인 정보 유출 공지 안내 이메일 위장 한 가상 화폐 채굴 악 성 코드 가 나타났 ...
2     행정 국가 기록원 일 국민 열람 수요 많 은 서울 경기 권 지적 원 만 건 공공누리...
3     소비자 잡지 초이스 추천 제품 선정 전자 건조기 호주 호평 받 았 다 전자 대표 조...
4     스타크래프트 대회 韓 위 데이비드 처칠 캐나다 메모리얼 대학교 컴퓨터 공학 교수 스...
5     성균관 대 연구 팀 표지 논문 니켈 양극 소재 결정 구조 특성 발견 특정 원소 조절...
6     손건재 포스코 신임 대표 사진 는 일 포스코 만 차별 화 된 핵심 역량 확보 하 겠...
7     가 국내 통신사 중 유일 하 게 아시아 최대 이동 통신 박람회 인 상하 이 에 참가...
8     반도체 경쟁력 강화 무역 장벽 완화 노력 대외 환경 변화 따른 우리나라 주요 대응 ...
9     하이닉스 지난 일 임직원 설 명회 개최 해 주 최대 시간 근무 제 시범 운영 유연근...
10    보도 합니다 손등 위 얇 은 패치 심장 박동 확인 해 로 건강 상태 알려 줍니다 피...
11    팹 리스 기업 인 지니 틱 스 는 산업 통상 자원 부 중소 벤처 기업 부 한국 산업...
12    주 는 자사 인공지능 서비스 에이브릴 보유 한 솔루션 산업 별 활용 사례 여종 담 ...
13    출고가 만 원 자기 부상 디자인 채용 스피커 경쟁 디자인 옮아간다 전자 대표 조성진...
14    모바일 플랫폼 서비스 회사 라인 플러스 대표 신중 호 는 블록체인 기술 전문 자회사...
15    사진 서울산 업진 흥원 제공 서울시 핵심 사업 수행 담당 하 는 서울산 업진 흥원 ...
16    한국 중견 기업 연합회 중견 기업 대한 합리 적 인식 확산 하 고 바람직 한 기업 ...
17    지난 월 일 전 구간 데이터 무제 한 시대 선언 하 며 출시 한 데이터 요금

### 2. 토픽 모델링

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [14]:
my_tokenizer = mecab.pos

In [15]:
tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(tokenized_docs)

# 모델 선언
lda = LatentDirichletAllocation(n_components=20, max_iter=20, learning_method='online')
lda_output = lda.fit_transform(tfidf)

### 3. 시각화

In [ ]:
!pip install pyLDAvis

In [17]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vect, mds='tsne')
pyLDAvis.display(vis)